In [1]:
# !mkdir html
# !mv *.html html
# !zip -r html.zip html

In [2]:
# !mkdir pdf

In [3]:
from glob import glob
from bs4 import BeautifulSoup
import os
import requests
from tqdm import tqdm
import random
import time
import json

In [4]:
cookies = {
    'from_cache': '',
    'auvid': 'MTY4MzQzNzkyMjk4NDowLjk0ODU3MDk4NDg1NDQ%3D',
    '_ga': 'GA1.2.534798164.1683437924',
    '_gid': 'GA1.2.1815999583.1683437924',
    'overridden_user_tests': '%7B%22google_account_permissions_v2%22%3A%22without_contacts%22%7D',
    'user_id': '38068128',
    'cookie_test': '38068128',
    'long_term_login': 'true',
    'login_token': '38068128%3Ba1784e11e7d9103b315e841f',
    '_sharedid': 'bdab44d3-8c83-4355-86f4-75591986fab6',
    '__qca': 'I0-518466091-1683439461266',
    '_pbjs_userid_consent_data': '3524755945110770',
    'ki_r': 'aHR0cHM6Ly9tYWxheWEuYWNhZGVtaWEuZWR1Lw%3D%3D',
    '_lr_env_src_ats': 'false',
    'pbjs-unifiedid': '%7B%22TDID%22%3A%2259a14f4e-9cd9-4559-947d-09f703acfc99%22%2C%22TDID_LOOKUP%22%3A%22TRUE%22%2C%22TDID_CREATED_AT%22%3A%222023-04-07T07%3A58%3A08%22%7D',
    'admin_unrecorded_tests': '%7B%22auth_system_version%22%3A%7B%22bucket%22%3A%22login_token_only%22%2C%22buckets%22%3A%5B%22y_cookie%22%2C%22login_token_only%22%5D%7D%2C%22localize_frontend%22%3A%7B%22bucket%22%3A%22localize%22%2C%22buckets%22%3A%5B%22control%22%2C%22localize%22%5D%7D%7D',
    'ab_tests': '%7B%22all_premium_general_modal_2023_04%22%3A%22control%22%2C%22profile_page_related_authors%22%3A%22related_authors%22%2C%22all_premium_faculty_modal_2023_04%22%3A%22faculty_image%22%7D',
    'ki_t': '1683446284799%3B1683446284799%3B1683447252094%3B1%3B9',
    'last_load_time': '1683447252',
    'cto_bundle': 'GubNql95MDE2bDVjUzF0eEJlbmwlMkI0dkxFSkUlMkYxWFJWMG9UZFZCMzU5YVBnJTJCRXFTZWhVS0ElMkY0ZjhjU3A5YlJvVGJDc29rJTJCbVJVS0UxWkYwbTdEQ3ZBRWI4dFdrcWVQSjJUdVZxUVdPejFHNVplZGxwcnF0WUpTdDlZc3pFNFc5ZlNXQXcxUm8xViUyRiUyRkh3N3lLZkhwanNBZGpCQSUzRCUzRA',
    'cto_bidid': 'Mtch719PcHRRZlpxVWhxZnlra0w1RlBvSzllUlRYYiUyQnAzMTFZMnJzQzZSd1pqZ1NnNWM5QnNsJTJCb21iUXQ0WERkeUVzTDBwMWFmNTFEem5XVkxSSUtqYyUyQmU5cm9NWldXRHdyNmwxSXFndGprQ1pkdyUzRA',
    'yauic': 'eyJfcmFpbHMiOnsibWVzc2FnZSI6IkJBaEpJaVl6T0RBMk9ERXlPRHRoTVRjNE5HVXhNV1UzWkRreE1ETmlNekUxWlRnME1XWUdPZ1pGVkE9PSIsImV4cCI6IjIwMjQtMDUtMDdUMDg6MTU6MDIuNjAyWiIsInB1ciI6bnVsbH19--5e1b19b4da42a13efc7ab718556d30a13085494b',
    '_cookie_session': 'WXdhaHVCSUpidnRzRUFhR1ZDNjZxaTlnVGtTeVhsT2c1WGtvR0tqUHZHQTIrS3k0TU1yQTF0a0Z4SCsxb0ZXUUhaSy83aFcvUkI1UjhVdmZ1b0VFWDBWd2hhRkM3Z1IvWVN0NFZrQkdtV1VzaDJvdHJtdXFycHVVUGY1bVFPeGIvcG1MM3dua3pkOFN1cEFoTE5Ub3VWdU9IVzV4aTVNbFVWYVRjdWdBMThjRTMyRXlsZVhGTlBWT0VJUVFCRFVST1NtVmhvNUQ0TDJldWlSQm1zYzd3ODRYczR1QUVoaGJSenJXTG82RGlwWUg2T3krektybnA0WHc1eFR5TWtzaTlyZlJPRm9DZkJZSDgrTjFqRjV5QUJ4bFIyQ0E3YWZGRk9JenEvSU95MlZleU4ybTFsVEZhTzBIZ0gvaWM0UWR2eUlvY1dJRE1wbGhtdFUyM1A1bjE5dlovV3NObHlyQk16eFlMR09ITEozVWN2bnM5QnVsc2JodDYraVB4eGM2NEZ4ZWViWWZWWXdQekdsMzBPUGNOK0QzQ0pKMEVhY0NGZXJHdGhXZTcyQnI0dG8wMDJjZ0FDTzV3U25ZL2VWa2RrRlh4Z2UyQXQ2UnJaUkNZZ05TeW1hQmhkdU1qUWxPVklnaTNaa1RrRnM9LS1vOTVoZkRLN0hndlVqYTB0dDRsekJRPT0%3D--ba9351aec1797d1e6f9bb70695ed88daa246b18e',
}

headers = {
    'authority': 'apiitmalaysia.academia.edu',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7',
    'cache-control': 'no-cache',
    # 'cookie': 'from_cache=; auvid=MTY4MzQzNzkyMjk4NDowLjk0ODU3MDk4NDg1NDQ%3D; _ga=GA1.2.534798164.1683437924; _gid=GA1.2.1815999583.1683437924; overridden_user_tests=%7B%22google_account_permissions_v2%22%3A%22without_contacts%22%7D; user_id=38068128; cookie_test=38068128; long_term_login=true; login_token=38068128%3Ba1784e11e7d9103b315e841f; _sharedid=bdab44d3-8c83-4355-86f4-75591986fab6; __qca=I0-518466091-1683439461266; _pbjs_userid_consent_data=3524755945110770; ki_r=aHR0cHM6Ly9tYWxheWEuYWNhZGVtaWEuZWR1Lw%3D%3D; _lr_env_src_ats=false; pbjs-unifiedid=%7B%22TDID%22%3A%2259a14f4e-9cd9-4559-947d-09f703acfc99%22%2C%22TDID_LOOKUP%22%3A%22TRUE%22%2C%22TDID_CREATED_AT%22%3A%222023-04-07T07%3A58%3A08%22%7D; admin_unrecorded_tests=%7B%22auth_system_version%22%3A%7B%22bucket%22%3A%22login_token_only%22%2C%22buckets%22%3A%5B%22y_cookie%22%2C%22login_token_only%22%5D%7D%2C%22localize_frontend%22%3A%7B%22bucket%22%3A%22localize%22%2C%22buckets%22%3A%5B%22control%22%2C%22localize%22%5D%7D%7D; ab_tests=%7B%22all_premium_general_modal_2023_04%22%3A%22control%22%2C%22profile_page_related_authors%22%3A%22related_authors%22%2C%22all_premium_faculty_modal_2023_04%22%3A%22faculty_image%22%7D; ki_t=1683446284799%3B1683446284799%3B1683447252094%3B1%3B9; last_load_time=1683447252; cto_bundle=GubNql95MDE2bDVjUzF0eEJlbmwlMkI0dkxFSkUlMkYxWFJWMG9UZFZCMzU5YVBnJTJCRXFTZWhVS0ElMkY0ZjhjU3A5YlJvVGJDc29rJTJCbVJVS0UxWkYwbTdEQ3ZBRWI4dFdrcWVQSjJUdVZxUVdPejFHNVplZGxwcnF0WUpTdDlZc3pFNFc5ZlNXQXcxUm8xViUyRiUyRkh3N3lLZkhwanNBZGpCQSUzRCUzRA; cto_bidid=Mtch719PcHRRZlpxVWhxZnlra0w1RlBvSzllUlRYYiUyQnAzMTFZMnJzQzZSd1pqZ1NnNWM5QnNsJTJCb21iUXQ0WERkeUVzTDBwMWFmNTFEem5XVkxSSUtqYyUyQmU5cm9NWldXRHdyNmwxSXFndGprQ1pkdyUzRA; yauic=eyJfcmFpbHMiOnsibWVzc2FnZSI6IkJBaEpJaVl6T0RBMk9ERXlPRHRoTVRjNE5HVXhNV1UzWkRreE1ETmlNekUxWlRnME1XWUdPZ1pGVkE9PSIsImV4cCI6IjIwMjQtMDUtMDdUMDg6MTU6MDIuNjAyWiIsInB1ciI6bnVsbH19--5e1b19b4da42a13efc7ab718556d30a13085494b; _cookie_session=WXdhaHVCSUpidnRzRUFhR1ZDNjZxaTlnVGtTeVhsT2c1WGtvR0tqUHZHQTIrS3k0TU1yQTF0a0Z4SCsxb0ZXUUhaSy83aFcvUkI1UjhVdmZ1b0VFWDBWd2hhRkM3Z1IvWVN0NFZrQkdtV1VzaDJvdHJtdXFycHVVUGY1bVFPeGIvcG1MM3dua3pkOFN1cEFoTE5Ub3VWdU9IVzV4aTVNbFVWYVRjdWdBMThjRTMyRXlsZVhGTlBWT0VJUVFCRFVST1NtVmhvNUQ0TDJldWlSQm1zYzd3ODRYczR1QUVoaGJSenJXTG82RGlwWUg2T3krektybnA0WHc1eFR5TWtzaTlyZlJPRm9DZkJZSDgrTjFqRjV5QUJ4bFIyQ0E3YWZGRk9JenEvSU95MlZleU4ybTFsVEZhTzBIZ0gvaWM0UWR2eUlvY1dJRE1wbGhtdFUyM1A1bjE5dlovV3NObHlyQk16eFlMR09ITEozVWN2bnM5QnVsc2JodDYraVB4eGM2NEZ4ZWViWWZWWXdQekdsMzBPUGNOK0QzQ0pKMEVhY0NGZXJHdGhXZTcyQnI0dG8wMDJjZ0FDTzV3U25ZL2VWa2RrRlh4Z2UyQXQ2UnJaUkNZZ05TeW1hQmhkdU1qUWxPVklnaTNaa1RrRnM9LS1vOTVoZkRLN0hndlVqYTB0dDRsekJRPT0%3D--ba9351aec1797d1e6f9bb70695ed88daa246b18e',
    'pragma': 'no-cache',
    'referer': 'https://apiitmalaysia.academia.edu/',
    'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
}

# response = requests.get(
#     'https://apiitmalaysia.academia.edu/Departments/Accounting_and_Finance/Documents?page=1',
#     cookies=cookies,
#     headers=headers,
# )

In [5]:
files = glob('department/*.json')
len(files)

32

In [6]:
with open(files[0]) as fopen:
    data = json.load(fopen)

In [7]:
data[0]

{'href': 'https://iium.academia.edu/Departments/AIKOL/Documents',
 'doc_count': 10}

In [139]:
for html in htmls:
    
    filename = html.split(') ')[1].replace(' - Academia.edu.html', '').replace(' ', '-')
    with open(html) as fopen:
        soup = BeautifulSoup(fopen.read(), "lxml")
    
    lis = soup.find_all('li', {'class': 'u-lineHeight2_5'})
    lis = [li for li in lis if ' documents</a>' in str(li) or ' document</a>' in str(li)]
    l = []
    for li in lis:
        href = li.find('a').get('href')
        try:
            doc_count = int(str(li).split(' documents</a>')[0].split('">')[-1])
        except:
            doc_count = 1
        l.append({
            'href': href,
            'doc_count': doc_count
        })
    
    with open(f'{filename}.json', 'w') as fopen:
        json.dump(l, fopen)

In [142]:
# !mkdir department
# !mv *.json department
# !zip -r department.zip department

In [129]:
len(lis)

227

In [130]:
lis[1]

<li class="u-lineHeight2_5"><a class="u-fw700 u-tcGrayDarkest" href="https://uitmshahalam.academia.edu/Departments/Aagbs/Documents">Aagbs</a><span class="u-tcGrayDark u-fs12">  <a class="u-tcGrayDark" href="https://uitmshahalam.academia.edu/Departments/Aagbs">78 people</a>  |  <a class="u-tcGrayDark" href="https://uitmshahalam.academia.edu/Departments/Aagbs/Documents">32 documents</a></span><span class="u-tcGrayDark u-fs12">  |  <span class="create-moderation-request clickable" data-confirm="Are you sure you want to hide Aagbs?" data-moderateable-id="662218" data-moderateable-type="Department" data-moderation-action="Hide">hide</span></span></li>

In [120]:
str(lis[0]).split(' document</a>')[0].split('">')[-1]

'1'

In [113]:
lis[0].find('a').get('href')

'https://uitmshahalam.academia.edu/Departments/Administrative/Documents'

In [15]:
soup = BeautifulSoup(response.content, "lxml")

In [18]:
scripts = soup.find_all('script')

In [36]:
data = []
for s in scripts:
    try:
        s = str(s).replace(',\n    })\n  }\n})();</script>', '').split('\n      work: ')[1]
        data.append(json.loads(s))
    except:
        pass

In [73]:
def is_downloadable(url):
    h = requests.head(url, cookies=cookies,
    headers=headers,allow_redirects=True)
    header = h.headers
    content_type = header.get('content-type')
    if 'text' in content_type.lower():
        return False
    if 'html' in content_type.lower():
        return False
    return h

def download_file_cloud(url, filename):
    r = requests.get(url, cookies=cookies,
    headers=headers,stream=True)
    total_size = int(r.headers['content-length'])
    version = int(r.headers.get('X-Bz-Upload-Timestamp', 0))
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'wb') as f:
        for data in r.iter_content(chunk_size=1_048_576):
            f.write(data)
    return version

In [57]:
data[0]['downloadable_attachments']

[{'id': 62378068,
  'asset_id': 42228543,
  'asset_type': 'Work',
  'always_allow_download': False,
  'scribd_thumbnail_url': 'https://imgv2-2-f.scribdassets.com/img/document/451860188/111x142/3d98e50758/1584347747?v=1',
  'download_url': 'https://d1wqtxts1xzle7.cloudfront.net/62378068/International_Journal_of_Marketing__Harisai_Doshi20200316-53074-g1kbq2-libre.pdf?1584431320=&response-content-disposition=attachment%3B+filename%3DAn_Application_of_Marketing_Mix_Strategy.pdf&Expires=1682773616&Signature=eYBII62SbL06~SDV~JPEkxXOm7u1dgC15oCiYLXEUMWN~IuvnI0VWbJhOHgVbC~Z3vumDNMyIzQ4mbQHUIvMW6xlTPvFTuQ11IE~~2Tlmz4DGuum3feTkMSsfvY7l4yJ2gO--YyLEOo4bhudBssMlN4GPTAqFFpOXX5KM0o0s066VbJsySn27Moc6X~wZzxFZIPv7iZXa0qjcatgN~zk3E3qyt7fmY9ph5voxix7jlcESZPpsGnOzeF6NbaFrgvtCr9B8v2x7eNpcMeMjBoQgYmpFNcSMepTLMKAyR5wHn2bDzSXFEhpwvZ0BhT7SYKyOVGpjkod3taDIMIZxy~j8g__&Key-Pair-Id=APKAJLOHF5GGSLRBV4ZA',
  'download_file_url': 'https://www.academia.edu/attachments/62378068/download_file?st=MTY4MzQ1NDc0Myw0Mi4xOTEuO

In [88]:
for i in tqdm(range(len(data))):
    try:
        url = data[i]['downloadable_attachments'][0]['download_file_url']
        h = is_downloadable(url)
        filename = h.headers['Content-Disposition'].split('filename=')[1]
        filename = os.path.join('pdf', filename)
        if not os.path.exists(filename):
            download_file_cloud(url, filename)
            data[i]['filename'] = filename

        time.sleep(random.uniform(1.5, 3.0))
    except Exception as e:
        print(e)

 55%|███████████████████████████████████████████████████████                                             | 11/20 [00:10<00:08,  1.04it/s]

list index out of range


 85%|█████████████████████████████████████████████████████████████████████████████████████               | 17/20 [00:33<00:09,  3.15s/it]

list index out of range


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:35<00:00,  1.78s/it]
